In [ ]:
import dxpy # dxpy allows python to interact with the platform storage
import numpy as np
import pandas as pd
import re
import subprocess
import glob
import os

# Load dataset descriptions
Output: entity_dictionary.csv, data_dictionary.csv, coding_dictionary.csv \
Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run


In [ ]:
## Load dataset descriptions

# dxpy.find_one_data_object() searches for an object of type 'Dataset' in the root folder '/' whose name matches the wildcard pattern 'app*.dataset'.
# And returns a dictionary describing the metadata of that dataset object.
dispensed_dataset = dxpy.find_one_data_object(typename='Dataset', name='app*.dataset', folder='/', name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

# dxpy.find_one_project() finds the current DNAnexus project, and returns a dictionary describing the metadata of the project.
project_id = dxpy.find_one_project()['id']

# Get DNAnexus dataset path 'project-xxxx:dataset-xxxx'
dataset = (':').join([project_id, dispensed_dataset_id])

# Extract the entire dataset and create three dictionary files that describe it:
# entity_dictionary.csv that contains the list of entities (tables) available. There is a participant table that contains information per participant.
# data_dictionary.csv that contains the list of fields (UK Biobank variable IDs) and which entity (table) they belong to.
# coding_dictionary.csv that contains a lookup table mapping coded numeric values to their human-readable meanings for some of the fields.

cmd = ['dx', 'extract_dataset', dataset, '-ddd', '--delimiter', ',']
subprocess.check_call(cmd)

# Load cohorts

In [ ]:
## Load cohorts

# Search for objects with criteria, take the first object, extract its unique DNAnexus object ID 'object-xxxx'
dispensed_control_id = list(
    dxpy.find_data_objects(typename='CohortBrowser', folder='/My/Cohorts', name_mode='exact', name='ischemic_controls',)
)[0]['id']
dispensed_case_id = list(
    dxpy.find_data_objects(typename='CohortBrowser', folder='/My/Cohorts', name_mode='exact', name='ischemic_cases',)
)[0]['id']

# Get DNAnexus path for cohort 'project-xxxx:object-xxxx'
control_dataset = (':').join([project_id, dispensed_control_id])
case_dataset = (':').join([project_id, dispensed_case_id])

# Retrieve phenotype data as a dataframe
Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run

In [ ]:
## Retrieve phenotype data
field_ids = [
    '31', # Sex
    '2966', # Age high blood pressure diagnosed
    '22001', # Genetic sex
    '22006', # Genetic ethnic grouping
    '22019', # Sex chromosome aneuploidy
    '22021', # Genetic kinship to other participants
    '21022', # Age at recruitment
    '22027', # Outliers for heterozygosity or missing rate
    '23104', # Body mass index (BMI)
    '20160', # Ever smoked
    '30760', # HDL cholesterol
    '30780', # LDL direct
    '22020', # Used in genetic principal components
    '22009' # Genetic principal components
]

# Load data_dictionary.csv into a Pandas dataframe
path = os.getcwd()
data_dict_csv = glob.glob(os.path.join(path, '*.data_dictionary.csv'))[0]
data_dict_df = pd.read_csv(data_dict_csv)

data_dict_df.head()

def fields_for_id(field_id):
    '''Collect all field names (e.g. 'p<field_id>_iYYY_aZZZ') given a list of field IDs and return string to pass into extract_dataset'''
    field_names = ['eid'] # participant id should be included
    # Loop through every field ID, use regex to find col names corresponding to the field ID, extract them as a list 
    for _id in field_id:
        select_field_names = list(
            data_dict_df[
                data_dict_df.name.str.match(r'^p{}(_i\d+)?(_a\d+)?$'.format(_id))
            ].name.values
        )
        # For field '22009' PCA, select the first 10 PCs
        if _id == '22009':
            field_names += select_field_names[:10]
        # For all fields except '2966' Age high blood pressure diagnosed, select only the first instance
        elif _id != '2966' and len(select_field_names) > 1:
            field_names += select_field_names[:1]
        # For field '2966' Age high blood pressure diagnosed, select all instances
        else:
            field_names += select_field_names
		# Qualify each field name with participant data table -> ['participant.eid', 'participant.p31_i0_a0', ...]
    field_names = [f'participant.{f}' for f in field_names] 
    # Combine all field names into one comma-delimited string so it can be passed to extract_dataset, return the string -> 'participant.eid,participant.p31_i0_a0,...'
    return ','.join(field_names)
    
field_names = fields_for_id(field_ids) 
field_names 

print(field_names) # There should be no space separating the different fields in the string

# Extract participant-level data for selected fields as control_dictionary.csv and case_dictionary.csv
cmd = ['dx', 'extract_dataset', control_dataset, 
		'--fields', field_names, 
		'--delimiter', ',', 
		'--output', 'control_dictionary.csv',
]
subprocess.check_call(cmd)
cmd = ['dx', 'extract_dataset', case_dataset,
    '--fields', field_names,
    '--delimiter', ',',
    '--output', 'case_dictionary.csv',
]
subprocess.check_call(cmd)

# Convert to Pandas dataframe, removes that prefix in col headers using regex substitution
# Note: re.sub('participant.', '', x) in original code is not precise. Unescaped dot represents 'any character' in regex.
control_df = pd.read_csv('control_dictionary.csv')
control_df = control_df.rename(columns=lambda x: re.sub(r'^participant\.', '', x)) 
case_df = pd.read_csv('case_dictionary.csv')
case_df = case_df.rename(columns=lambda x: re.sub(r'^participant\.', '', x)) 

# Create target phenotype variable, combine cases and controls into a single Pandas dataframe
case_df['ischemia_cc'] = 1
control_df['ischemia_cc'] = 0
df = pd.concat([case_df, control_df])

df.head()
print(df.shape)
df.ischemia_cc.value_counts() # The counts should be consistent with the counts from Cohort Browser

# Sample QC and imputation

In [ ]:
## Sample QC and imputation

df_qced = df[
    (df['p31'] == df['p22001']) # Filter reported sex and genetic sex are the same
    & (df['p22006'] == 1)  # Only include Caucasian ancestry (In_white_british_ancestry_subset)
    & (df['p22019'].isnull()) # No Sex chromosome aneuploidy
    & (df['p22020'] == 1)  # Participant was used to calculate PCA 
    & (df['p22027'].isnull())    # exclude samples flagged as heterozygosity or missingness outliers
].copy()

df_qced.head()
df_qced.ischemia_cc.value_counts()

# Aggregate multiple instances of '2966' Age high blood pressure diagnosed into one binary variable
df_qced['hypertension'] = [
    np.nansum([float(row[i]) for i in [0, 1, 2, 3]]) > 0
    for row in df_qced[['p2966_i0', 'p2966_i1', 'p2966_i2', 'p2966_i3']].to_numpy()
]
df_qced['hypertension'] = df_qced['hypertension'].astype(int)

# Impute missing values for '20160' Ever smoked as 0
# Note: df_qced['p20160_i0'].fillna(0, inplace=True) in original code get a view (not copy) of the df slice, inplace=True may not modify the df slice as intended.
df_qced['p20160_i0'] = df_qced['p20160_i0'].fillna(0)

# Impute missing values for '23104' BMI, '30760' HDL cholesterol, '30780' LDL direct with mean value
df_qced['p23104_i0'] = df_qced['p23104_i0'].fillna(df_qced['p23104_i0'].mean())
df_qced['p30760_i0'] = df_qced['p30760_i0'].fillna(df_qced['p30760_i0'].mean())
df_qced['p30780_i0'] = df_qced['p30780_i0'].fillna(df_qced['p30780_i0'].mean())

# Check that there are no missing values
df_qced.isna().sum()

# Rename PC cols and core phenotype cols
df_qced = df_qced.rename(columns=lambda x: re.sub('p22009_a','pc',x))
df_qced = df_qced.rename(
    columns={
        'eid': 'IID',
        'p31': 'sex',
        'p21022': 'age',
        'p20160_i0': 'ever_smoked',
        'p23104_i0': 'bmi',
        'p30760_i0': 'hdl_cholesterol',
        'p30780_i0': 'ldl_cholesterol',
    }
)

# Add FID column, required input format for PLINK and Regenie
df_qced['FID'] = df_qced['IID']

# Create a file for target phenotype and covariates that can be used in Regenie (--phenoFile/--covarFile)
cols = ['FID', 'IID', 'sex', 'age', 'bmi', 'ever_smoked', 'hdl_cholesterol', 'ldl_cholesterol', 'hypertension', 'ischemia_cc',]
cols.extend([col for col in df_qced if 'pc' in col])
df_phenotype = df_qced[cols]

df_phenotype.head()

# Select only samples with genotype imputation available

In [ ]:
## Select only samples with genotype imputation available
# Intersect the phenotype file and the imputation sample file based on IID, drop redundant cols
# Note: Original code reads c1.sample file. It is changed to c22.sample here to save memory.
# Note: sep='\s' in original code is not standard python escape sequence. Use the raw string r'\s+' instead.
path_to_impute_file = f'/mnt/project/Bulk/Imputation/Imputation from genotype (GEL)/ukb21008_c22_b0_v1.sample' 
sample_file = pd.read_csv(path_to_impute_file, sep=r'\s+', header=0, names=['FID', 'IID', 'missing', 'sex'], engine='python',)

ischemia_df = df_phenotype.join(sample_file.set_index('IID'), on='IID', rsuffix='_sample', how='inner')
ischemia_df.drop(columns=['FID_sample', 'missing', 'sex_sample'], axis=1, inplace=True, errors='ignore')

# Write phenotype file to a tsv file with suffix .phe and upload to RAP

In [ ]:
## Write phenotype files to a TSV file ischemia_df.phe and upload to RAP
ischemia_df.to_csv('ischemia_df.phe', sep='\t', na_rep='NA', index=False, quoting=3)
ischemia_df.head()

output_dir = '/My/Data/'

subprocess.run([
    "dx", "upload", "ischemia_df.phe",
    "-p",
    "--path", output_dir,
    "--brief"
])

# Original code used a bash cell magic to upload the file:
# %%bash -s "$output_dir"
# dx upload ischemia_df.phe -p --path $1 --brief